# Cleanup All Resources

This notebook is designed to remove all resources by the type created.  By being a general cleanup utility, you can skip sections that you want to remain in use.  This notebook also has a section to delete the GCS bucket created in the `00 - Environment Setup` notebook.

### Resources
- Vertex AI: https://googleapis.dev/python/aiplatform/latest/index.html
- BigQuery: https://googleapis.dev/python/bigquery/latest/index.html
- GCS: https://googleapis.dev/python/storage/latest/client.html

---
## Method 1: Delete The Project
- [Instructions](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)
    - Open Console > IAM & Admin > Settings: [Link](https://console.cloud.google.com/iam-admin/settings)
    - Click Select a project.
    - Select a project you want to delete, and click Open.
    - Click Shut down.
    - Enter the Project ID, then click Shut down.

---
## Method 2: Remove the Items Created By This Notebook

## Setup
When Finished with removing each Vertex AI, GCS, and BigQuery item below, you will then need to stop and delete this notebook instance in Console > Vertex AI > Notebooks.

inputs:

In [1]:
PROJECT_ID = "statmike-mlops"
REGION = 'us-central1'

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

packages:

In [3]:
from google.cloud import aiplatform
from google.cloud import aiplatform_v1beta1
from google.cloud import bigquery
from google.cloud import storage

parameters:

In [4]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

clients:

In [5]:
aiplatform.init(project=PROJECT_ID, location=REGION)
clients['bq'] = bigquery.Client(project=PROJECT_ID)
clients['fs'] = aiplatform_v1beta1.FeaturestoreServiceClient(client_options=client_options)
clients['viz'] = aiplatform_v1beta1.VizierServiceClient(client_options=client_options)
clients['tb'] = aiplatform_v1beta1.TensorboardServiceClient(client_options=client_options)
clients['md'] = aiplatform_v1beta1.MetadataServiceClient(client_options=client_options)
clients['gcs'] = storage.Client()

---
## BigQuery
- Delete the dataset `digits`

list of datasets in the project:

In [6]:
datasets = list(clients['bq'].list_datasets())

delete each dataset:

In [7]:
for ds in datasets:
    print(ds.dataset_id)
    clients['bq'].delete_dataset(ds.dataset_id,delete_contents=True,not_found_ok=True)

digits


---
## Vertex AI > Datasets
- Delete datasets

In [10]:
datasets = aiplatform.TabularDataset.list()

In [11]:
for ds in datasets:
    print(ds.resource_name)
    aiplatform.TabularDataset(dataset_name=ds.resource_name).delete()

projects/691911073727/locations/us-central1/datasets/8862828979967492096
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/8862828979967492096
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/3452940125850304512
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/8862828979967492096
projects/691911073727/locations/us-central1/datasets/459112075294146560
INFO:google.cloud.aiplatform.base:Deleting TabularDataset : projects/691911073727/locations/us-central1/datasets/459112075294146560
INFO:google.cloud.aiplatform.base:Delete TabularDataset  backing LRO: projects/691911073727/locations/us-central1/operations/7573733784894308352
INFO:google.cloud.aiplatform.base:TabularDataset deleted. . Resource name: projects/691911073727/locations/us-central1/datasets/459112075294146560
proje

---
## Vertex AI > Features
- delete feature store including entity types and features

In [13]:
fs = clients['fs'].list_featurestores(parent=PARENT)

In [14]:
for f in fs:
    print(f.name)
    clients['fs'].delete_featurestore(request = aiplatform_v1beta1.types.DeleteFeaturestoreRequest(name=f.name,force=True))

projects/691911073727/locations/us-central1/featurestores/digits


---
## Vertex AI > Pipelines
* Delete Managed Pipelines

In [15]:
pl = aiplatform.PipelineJob.list()

In [16]:
for p in pl:
    print(p.name)
    p.delete()

kfp-02c-digits-20210919213805-20210920012552
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210919213805-20210920012552
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/214851993770917888
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210919213805-20210920012552
kfp-02c-digits-20210916004500-20210916004518
INFO:google.cloud.aiplatform.base:Deleting PipelineJob : projects/691911073727/locations/us-central1/pipelineJobs/kfp-02c-digits-20210916004500-20210916004518
INFO:google.cloud.aiplatform.base:Delete PipelineJob  backing LRO: projects/691911073727/locations/us-central1/operations/6641488662028615680
INFO:google.cloud.aiplatform.base:PipelineJob deleted. . Resource name: projects/691911073727/locations/us-central1/pipelineJobs/kfp-02

---
## Vertex AI > Training
- delete various jobs types
- some create with aiplatform.* and other created by aiplatform.gapic.*

## Vertex AI > Training > Training Pipelines
- Delete training pipelines

In [17]:
jobs = aiplatform.CustomTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

05c_digits_20210919214125
INFO:google.cloud.aiplatform.base:Deleting CustomTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/3736564922671169536
INFO:google.cloud.aiplatform.base:Delete CustomTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/4767991217042489344
INFO:google.cloud.aiplatform.base:CustomTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/3736564922671169536
05d_digits_20210919123929
INFO:google.cloud.aiplatform.base:Deleting CustomTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/521135526216990720
INFO:google.cloud.aiplatform.base:Delete CustomTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/5520092354813362176
INFO:google.cloud.aiplatform.base:CustomTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/521135526216990720
05d_digits_20210919114251
INFO:google.cloud.

In [18]:
jobs = aiplatform.CustomPythonPackageTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

In [19]:
jobs = aiplatform.AutoMLTabularTrainingJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

02c_digits_20210919213805
INFO:google.cloud.aiplatform.base:Deleting AutoMLTabularTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/2019286089759457280
INFO:google.cloud.aiplatform.base:Delete AutoMLTabularTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/2187428630559195136
INFO:google.cloud.aiplatform.base:AutoMLTabularTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/2019286089759457280
02b_digits_20210919205707
INFO:google.cloud.aiplatform.base:Deleting AutoMLTabularTrainingJob : projects/691911073727/locations/us-central1/trainingPipelines/275548609036943360
INFO:google.cloud.aiplatform.base:Delete AutoMLTabularTrainingJob  backing LRO: projects/691911073727/locations/us-central1/operations/5772293933946109952
INFO:google.cloud.aiplatform.base:AutoMLTabularTrainingJob deleted. . Resource name: projects/691911073727/locations/us-central1/trainingPipelines/275548609036943360
02

## Vertex AI > Training > Custom Job
- Delete custom training jobs

In [20]:
jobs = aiplatform.CustomJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

caip_pipelines_7436225837067665408_5190583288497963008
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/7441057091160113152
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/4493271639772889088
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/7441057091160113152
caip_pipelines_7436225837067665408_-8644474766784200704
INFO:google.cloud.aiplatform.base:Deleting CustomJob : projects/691911073727/locations/us-central1/customJobs/9149610199793795072
INFO:google.cloud.aiplatform.base:Delete CustomJob  backing LRO: projects/691911073727/locations/us-central1/operations/9104957658200276992
INFO:google.cloud.aiplatform.base:CustomJob deleted. . Resource name: projects/691911073727/locations/us-central1/customJobs/9149610199793795072
caip_pipelines_7436225837067665408_-4032788748356812800
INF

## Vertex AI > Training > Hyperparameter Tuning Job

In [21]:
jobs = aiplatform.HyperparameterTuningJob.list()
for j in jobs:
    print(j.display_name)
    j.delete()

05f_digits_20210920145828
INFO:google.cloud.aiplatform.base:Deleting HyperparameterTuningJob : projects/691911073727/locations/us-central1/hyperparameterTuningJobs/6687856441761464320
INFO:google.cloud.aiplatform.base:Delete HyperparameterTuningJob  backing LRO: projects/691911073727/locations/us-central1/operations/1687529071921070080
INFO:google.cloud.aiplatform.base:HyperparameterTuningJob deleted. . Resource name: projects/691911073727/locations/us-central1/hyperparameterTuningJobs/6687856441761464320
05e_digits_20210920125450
INFO:google.cloud.aiplatform.base:Deleting HyperparameterTuningJob : projects/691911073727/locations/us-central1/hyperparameterTuningJobs/6831945241558253568
INFO:google.cloud.aiplatform.base:Delete HyperparameterTuningJob  backing LRO: projects/691911073727/locations/us-central1/operations/5673214742143959040
INFO:google.cloud.aiplatform.base:HyperparameterTuningJob deleted. . Resource name: projects/691911073727/locations/us-central1/hyperparameterTuningJob

---
## Vertex AI > Experiments > Experiment & Tensorboard Instances

In [26]:
tbs = clients['tb'].list_tensorboards(parent=PARENT)

In [28]:
for tb in tbs:
    print(tb.name)
    tbes = clients['tb'].list_tensorboard_experiments(parent=tb.name)
    for tbe in tbes:
        print(tbe.name)
        clients['tb'].delete_tensorboard_experiment(request=aiplatform_v1beta1.types.DeleteTensorboardExperimentRequest(name=tbe.name))
    clients['tb'].delete_tensorboard(request = aiplatform_v1beta1.types.DeleteTensorboardRequest(name=tb.name))

projects/691911073727/locations/us-central1/tensorboards/8640050331994030080


## Vertex AI > Experiments > Studies
Vizier Studies

In [23]:
studies = clients['viz'].list_studies(parent=PARENT)

In [24]:
for study in studies:
    print(study.name)
    clients['viz'].delete_study(name=study.name)

projects/691911073727/locations/us-central1/studies/4214226082825
projects/691911073727/locations/us-central1/studies/639592116037


---
## Vertex AI > Endpoints
- Delete endpoints forcing deployed models to undeploy

In [29]:
endpoints = aiplatform.Endpoint.list()

In [30]:
for ep in endpoints:
    print(ep.resource_name)
    ep.delete(force=True)

projects/691911073727/locations/us-central1/endpoints/7936169376170901504
INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/691911073727/locations/us-central1/endpoints/7936169376170901504
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/691911073727/locations/us-central1/endpoints/7936169376170901504/operations/7231460213214150656
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/691911073727/locations/us-central1/endpoints/7936169376170901504
INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/691911073727/locations/us-central1/endpoints/7936169376170901504
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/691911073727/locations/us-central1/operations/6609963464637022208
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/691911073727/locations/us-central1/endpoints/7936169376170901504
projects/691911073727/locations/us-central1/endp

---
## Vertex AI > Models
- Delete endpoints first (contain deployed models)
- Delete uploaded models

In [6]:
models = aiplatform.Model.list()

In [7]:
for model in models:
    print(model.resource_name)
    model.delete()

projects/1026793852137/locations/us-central1/models/6133608023362371584
Deleting Model : projects/1026793852137/locations/us-central1/models/6133608023362371584
Delete Model  backing LRO: projects/1026793852137/locations/us-central1/operations/2080748308715798528
Model deleted. . Resource name: projects/1026793852137/locations/us-central1/models/6133608023362371584
projects/1026793852137/locations/us-central1/models/2891016291655614464
Deleting Model : projects/1026793852137/locations/us-central1/models/2891016291655614464
Delete Model  backing LRO: projects/1026793852137/locations/us-central1/operations/2970209235121471488
Model deleted. . Resource name: projects/1026793852137/locations/us-central1/models/2891016291655614464
projects/1026793852137/locations/us-central1/models/6928493357593264128
Deleting Model : projects/1026793852137/locations/us-central1/models/6928493357593264128
Delete Model  backing LRO: projects/1026793852137/locations/us-central1/operations/1432229962374447104


---
## Vertex AI > Batch Predictions
- delete all batch prediction jobs

In [33]:
bps = aiplatform.BatchPredictionJob.list()

In [34]:
for bp in bps:
    print(bp.resource_name)
    bp.delete()

---
## Vertex AI > Metadata

In [36]:
md = clients['md'].list_metadata_stores(parent=PARENT)

In [47]:
for m in md:
    print(m.name)
    arts = clients['md'].list_artifacts(parent=m.name)
    for a in arts:
        print(a.name)
        clients['md'].delete_artifact(name=a.name)
    if m.name.split('/')[-1] != 'default':
        print('delete would happen here - update code to make this happen')
        #clients['md'].get_metadata_store(name=m.name)#.purge_artifacts(parent=m.name)

projects/691911073727/locations/us-central1/metadataStores/default
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/6180199488504697479
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/1222243081411306850
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/873618663861831837
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/3239445619323675727
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/9832748077701759572
projects/691911073727/locations/us-central1/metadataStores/default/artifacts/8919405980845928163


---
## Storage Bucket Removal

- delete contents and storage bucket `statmike-mlops` (named after project)

In [50]:
bucket = clients['gcs'].get_bucket(PROJECT_ID)

In [ ]:
#bucket.delete(force=True)